# Jet 3D visualization

In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go

import os

## Functions

In [203]:
def read_data(fname, dir="../data/"):
    # Read data from csv file
    return pd.read_csv(os.path.join(dir, fname))

def rgba_to_hex(rgba):
    r, g, b, a = rgba
    if a == 1:
        return '#{:02x}{:02x}{:02x}'.format(int(r*255), int(g*255), int(b*255))
    else:
        return '#{:02x}{:02x}{:02x}{:02x}'.format(int(r*255), int(g*255), int(b*255), int(a*255))

## Constants

In [3]:
JET_FNAME      = "jet_df.csv"
PARTICLE_FNAME = "particle_df.csv"

FONTSIZE = 16

SAVE_FIG = False

## Code

### Read data

In [207]:
# read in the data
jet_df = read_data(JET_FNAME)
par_df = read_data(PARTICLE_FNAME)

### Randomly select a jet to visualize

In [215]:
# filter out events that has at least one jet with nParticles < 10
df = jet_df.groupby("eventID").filter(lambda x: np.all(x.nParticles >= 10))
# filter out events with only one jet
df = df.groupby("eventID").filter(lambda x: len(x) == 2)
# filter out events that has particles of particleType 0 that is in the par_df dataframe
df = df.groupby("eventID").filter(lambda x: np.all(par_df[par_df.eventID == x.eventID.unique()[0]].particleType != 0))

# randomly choose one eventID from the filtered data
ev_id = np.random.choice(df.eventID.unique())
# ev_id = 945, 3614  # good example

print(f"Event ID: {ev_id}")

jet_df = jet_df[jet_df.eventID == ev_id]
par_df = par_df[par_df.eventID == ev_id]

print(f"Number of jets: {len(jet_df)}")
print(f"Number of particles: {jet_df.nParticles.values}")

Event ID: 3614
Number of jets: 2
Number of particles: [43 37]


### Visualize the jet

In [216]:
particles = pd.DataFrame(columns=["jetID", "p", "px", "py", "pz", "theta", "phi", "type"])

particles["jetID"] = par_df["jetID"]
particles["theta"] = par_df["particleTheta"]
particles["phi"]   = par_df["particlePhi"]
particles["type"]  = par_df["particleType"]

# compute the momentum 
particles["p"] = np.sqrt(par_df["particlePx"]**2 + par_df["particlePy"]**2 + par_df["particlePz"]**2)

particles["px"] = particles["p"]*np.sin(particles["theta"])*np.cos(particles["phi"])
particles["py"] = particles["p"]*np.sin(particles["theta"])*np.sin(particles["phi"])
particles["pz"] = particles["p"]*np.cos(particles["theta"])

particles.loc[:, "name"] = particles["type"].map({
    1:    "down",
    2:    "up",
    11:   "electron",
    -11:  "electron",
    13:   "muon",
    -13:  "muon",
    22:   "photon",
    130:  "K0L",
    211:  "pion",
    -211: "pion"
})

In [217]:
# use matplotlib tab10 colormap in hex format
jetID_colors = plt.cm.tab10(np.linspace(0, 1, 10))[[0, 1]]

jetID_colors = [rgba_to_hex(c) for c in jetID_colors]

data = []
for i, jetID in enumerate(particles['jetID'].unique()):
    mask = (particles['jetID'] == jetID)
    
    data += [
        go.Scatter3d(
            x=[0,particlePx], 
            y=[0,particlePy], 
            z=[0,particlePz],
            mode='lines',
            line=dict(color=jetID_colors[i], width=5),
            name=f"Jet {i+1}"
        ) 
        for particlePx,particlePy,particlePz in zip(particles[mask]['px'], particles[mask]['py'], particles[mask]['pz'])
    ]
    
layout = go.Layout(
    scene=dict(
        xaxis_title = '',
        yaxis_title = '',
        zaxis_title = '',
        xaxis=dict(showline=False, showticklabels=False, showgrid=False, range=(-1,1)),
        yaxis=dict(showline=False, showticklabels=False, showgrid=False, range=(-1,1)),
        zaxis=dict(showline=False, showticklabels=False, showgrid=False, range=(-1,1)),
        aspectmode='cube',
        aspectratio=dict(x=1, y=1, z=1),
    ),
    paper_bgcolor = 'white',
    plot_bgcolor  = 'white',
    width         = 500,
    height        = 500,
    showlegend    = False,
    margin        = dict(l=10, r=10, b=10, t=10),
)

fig = go.Figure(data=data, layout=layout)
fig.show()


In [221]:
# color code the particles by their type 
type_colors = plt.cm.tab10(np.linspace(0, 1, 10))



type_colors = [rgba_to_hex(c) for c in type_colors]

type_colors = {
    "pion":     type_colors[0], # blue
    "photon":   type_colors[1], # orange
    "K0L":      type_colors[2], # green
    "up":       type_colors[3], # red
    "down":     type_colors[4], # purple
    "muon":     type_colors[5], # brown
    "electron": type_colors[6], # pink
}

data = []
for p_type in particles['name'].unique():
    mask = (particles['name'] == p_type)
    data += [
        go.Scatter3d(
            x=[0,particlePx], 
            y=[0,particlePy], 
            z=[0,particlePz],
            mode='lines',
            line=dict(color=type_colors[p_type], width=3),
            name=f"{particles[mask]['name'].iloc[0]}"
        ) 
        for particlePx,particlePy,particlePz in zip(particles[mask]['px'], particles[mask]['py'], particles[mask]['pz'])
    ]

layout = go.Layout(
    scene=dict(
        xaxis_title = '',
        yaxis_title = '',
        zaxis_title = '',
        xaxis=dict(showline=False, showticklabels=False, showgrid=False, range=(-1,1)),
        yaxis=dict(showline=False, showticklabels=False, showgrid=False, range=(-1,1)),
        zaxis=dict(showline=False, showticklabels=False, showgrid=False, range=(-1,1)),
        aspectmode='cube',
        aspectratio=dict(x=1, y=1, z=1),
    ),
    paper_bgcolor = 'white',
    plot_bgcolor  = 'white',
    width         = 500,
    height        = 500,
    showlegend    = False,
    margin        = dict(l=10, r=10, b=10, t=10),
)

fig = go.Figure(data=data, layout=layout)
fig.show()
